<a target="_blank" href="https://colab.research.google.com/github/N-Hoffmann/sg-nex-data/blob/master/docs/bambu_tutorial.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# **Transcript discovery and quantification of SG-NEx samples**

In this tutorial, we will perform novel transcript discovery and
quantification on the SG-NEx samples. We will be using six Nanopore direct RNA-Sequencing
samples, three replicates each from the A549 and
HepG2 cell lines. The A549 cell line was extracted from lung tissues
from a patient with lung cancer whereas HepG2 was extracted from
hepatocellular carcinoma from a patient with liver cancer. We will use
Bambu, a R package hosted on the Bioconductor platform to identify and
quantify novel isoforms in these cell lines. 

**Note: This tutorial may take 10 minutes to complete.**


## **Content**

- [Installation](#installation)
- [Data Access and Preparation](#data-access-and-preparation) 
- [Running Bambu](#running-bambu)
- [Reference](#reference)


## **Installation**

First, we have to install Bambu. Before that, make sure you have R (version >= 4.1) installed on your machine. We can install Bambu using the following command:

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("bambu", update = FALSE)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.16 (BiocManager 1.30.20), R 4.2.3 (2023-03-15)

Installing package(s) 'BiocVersion', 'bambu'

also installing the dependencies ‘rjson’, ‘formatR’, ‘png’, ‘filelock’, ‘XML’, ‘restfulr’, ‘lambda.r’, ‘futile.options’, ‘KEGGREST’, ‘plogr’, ‘BiocFileCache’, ‘MatrixGenerics’, ‘Biobase’, ‘DelayedArray’, ‘Biostrings’, ‘rtracklayer’, ‘matrixStats’, ‘XVector’, ‘futile.logger’, ‘snow’, ‘BH’, ‘RCurl’, ‘GenomeInfoDbData’, ‘AnnotationDbi’, ‘RSQLite’, ‘BiocIO’, ‘biomaRt’, ‘zlibbioc’, ‘bitops’, ‘Rhtslib’, ‘SummarizedExperiment’, ‘S4Vectors’, ‘BSgenome’, ‘IRanges’, ‘BiocGenerics’, ‘BiocParallel’, ‘GenomeInfoDb’, ‘GenomicAlignments’, ‘GenomicFeatures’, ‘GenomicRanges’, ‘Rsamtools’, ‘Rcpp’, ‘xgboost’, ‘RcppArma

If you want a more recent version of Bambu, you may refer to the Bambu Github repository [here](https://github.com/GoekeLab/bambu). 


## **Data Access and Preparation**
### **Download Data for Bambu**
Next, we will need to download the required data to run Bambu. The required data include:

-   a set of aligned reads to the genome from the A549 and HepG2 cell lines (bam files),
-   reference human genome annotations (gtf file, TxDb object, or Bambu
    Annotation object),
-   reference human genome sequence (fasta file or BSgenome).

Generally, you may want to learn how to get access to these data using the [data
access
tutorial](AWS_data_access_tutorial.md). Below we only show the necessary steps to download the required data. The following command requires you to have [AWS CLI](https://aws.amazon.com/cli/) installed.

In [ ]:
# install AWS CLI
system("sudo apt install awscli")

In [ ]:
# create a directory to store the data
system("mkdir bambu_tutorial")
# download genome fasta file 
system("aws s3 cp --no-sign-request s3://sg-nex-data/data/data_tutorial/annotations/hg38_chr22.fa ./bambu_tutorial")
# download genome index fastai file 
system("aws s3 cp --no-sign-request s3://sg-nex-data/data/data_tutorial/annotations/hg38_chr22.fa.fai ./bambu_tutorial")
# download gtf file
system("aws s3 cp --no-sign-request s3://sg-nex-data/data/data_tutorial/annotations/hg38_chr22.gtf ./bambu_tutorial")
# download aligned bam files for A549 samples and HepG2 samples
system("aws s3 sync --no-sign-request s3://sg-nex-data/data/data_tutorial/bam ./bambu_tutorial --include *.bam ")

**NOTE: We have downsampled the Hg38 genome, A549 and HepG2 samples to ensure this tutorial can be completed in 10 minutes. If you want to run Bambu on the original samples, you can find the sample name [here](https://github.com/GoekeLab/sg-nex-data/blob/master/docs/samples.tsv) and amend it into the following code chunk:**

In [ ]:
# Note: Please make sure to replace the "sample_alias" with your sample name 
# To download genome bam files
system("aws s3 sync --no-sign-request s3://sg-nex-data/data/sequencing_data_ont/bam/genome/<sample_alias> ./bambu_tutorial")

### **Prepare Data for Bambu**

All required data are now stored in the `bambu_tutorial` folder of the
current working directory. Next, we prepare the data to run Bambu.

In [ ]:
# set work directory if you are in a different directory
setwd("bambu_tutorial")

# data preparation
library(bambu)
fa.file <- "hg38_chr22.fa"
gtf.file <- "hg38_chr22.gtf"
annotations <- prepareAnnotations(gtf.file) # This function creates a reference annotation object which is used for transcript discovery and quantification in Bambu.
samples.bam <- list.files(".", pattern = ".bam$", full.names = TRUE)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

## **Running Bambu**

Now we can run Bambu with these data. For a
faster running speed, you can increase the `ncore` parameter up
to the total number of samples at your availability. 

In [ ]:
# running Bambu 
se <- bambu(reads = samples.bam, annotations = annotations, genome = fa.file, ncore = 2)

WARNING - If you change the number of cores (ncore) between Bambu runs and there is no progress please restart your R session to resolve the issue that originates from the XGboost package.

--- Start generating read class files ---

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com



  |                                                                      |   0%[04:26:36] WARNING: src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[04:26:36] WARNING: src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[04:26:36] WARNING: src/learner.cc:553: 
  If you are loading a serialized mo

Detected 15 warnings across the samples during read class construction. Access warnings with metadata(bambuOutput)$warnings

--- Start extending annotations ---

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.
ℹ The deprecated feature was likely used in the bambu package.
  Please report the issue to the authors.”
Using a novel discovery rate (NDR) of: 0.129

--- Start isoform quantification ---

--- Finished running Bambu ---



Bambu returns a `SummarizedExperiment` object with the genomic
coordinates of the annotated & novel transcripts and their expression
estimates. They can be assessed using the following code:

In [1]:
assays(se) #returns the transcript abundance estimates as counts or CPM.

ERROR: ignored

In [ ]:
rowRanges(se) #returns a GRangesList (with genomic coordinates) with all annotated and newly discovered transcripts.

GRangesList object of length 4547:
$BambuTx1
GRanges object with 3 ranges and 2 metadata columns:
      seqnames            ranges strand | exon_rank exon_endRank
         <Rle>         <IRanges>  <Rle> | <integer>    <integer>
  [1]       22 15784968-15785057      + |         1            3
  [2]       22 15788820-15788931      + |         2            2
  [3]       22 15852990-15853424      + |         3            1
  -------
  seqinfo: 1 sequence from an unspecified genome; no seqlengths

$BambuTx2
GRanges object with 7 ranges and 2 metadata columns:
      seqnames            ranges strand | exon_rank exon_endRank
         <Rle>         <IRanges>  <Rle> | <integer>    <integer>
  [1]       22 15784971-15785057      + |         1            7
  [2]       22 15787172-15787282      + |         2            6
  [3]       22 15788552-15788699      + |         3            5
  [4]       22 15788820-15788931      + |         4            4
  [5]       22 15790661-15790798      + |        

In [ ]:
rowData(se) #returns additional information about each transcript such as the gene name and the class of the newly discovered transcript.

DataFrame with 4547 rows and 11 columns
                         TXNAME          GENEID        NDR novelGene
                    <character>     <character>  <numeric> <logical>
BambuTx1               BambuTx1 ENSG00000223875 0.00632911     FALSE
BambuTx2               BambuTx2 ENSG00000206195 0.11159737     FALSE
BambuTx3               BambuTx3 ENSG00000070010 0.01312336     FALSE
BambuTx4               BambuTx4 ENSG00000128185 0.03818616     FALSE
BambuTx5               BambuTx5 ENSG00000272779 0.00354610     FALSE
...                         ...             ...        ...       ...
ENST00000641859 ENST00000641859 ENSG00000284633         NA     FALSE
ENST00000641915 ENST00000641915 ENSG00000284665         NA     FALSE
ENST00000641933 ENST00000641933 ENSG00000284651         NA     FALSE
ENST00000641967 ENST00000641967 ENSG00000284630         NA     FALSE
ENST00000642075 ENST00000642075 ENSG00000284633         NA     FALSE
                novelTranscript     txClassDescription readCoun

This `SummarizedExperiment` object can also be used for further downstream analysis (eg. [DESeq](http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html)). If you want to save the transcript &  genomic annotations and their expression
estimates, you can then write them into an `output` folder using the `writeBambuOutput` function.

In [ ]:
writeBambuOutput(se, path = "./output")

The files in the `output` folder is described below:

| Output file name                | Description                                                             |
|:----------------------------|:------------------------------------------|
| extended_annotations.gtf        | Extended transcript & gene annotations for the genome using long reads data.        |
| counts_transcript.txt           | Total read counts estimates for each transcript in each sample.        |
| CPM_transcript.txt              | Counts per million (CPM) estimates for each transcript in each sample. |
| fullLengthCounts_transcript.txt | Full length read counts estimates for each transcript in each sample.  |
| uniqueCounts_transcript.txt                | Unique read counts estimates for each transcript in each sample.       |
| counts_gene.txt                 | Gene read counts estimates for each transcript in each sample.         |

**NOTE: This is a short tutorial to demonstrate the usage of Bambu on the SG-NEx data. Please refer to the [Bambu documentation](https://github.com/GoekeLab/bambu) for a more complete workflow in novel transcript discovery and quantification.**

## **Reference**

In this tutorial, we extended the existing transcript & gene annotations
on the [SGNEx](https://github.com/GoekeLab/sg-nex-data) dataset using
[Bambu](https://github.com/GoekeLab/bambu). If you use Bambu and the
dataset from SG-NEx in your work, please cite the following paper.

Chen, Ying, et al. “A systematic benchmark of Nanopore long read RNA
sequencing for transcript level analysis in human cell lines.” bioRxiv
(2021). doi: <https://doi.org/10.1101/2021.04.21.440736>